In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 9.6MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical, plot_model
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint
import kerastuner as kt
from keras.models import Model   
from keras.layers import * 

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
_ = tf.compat.v1.Session(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [4]:
from google.colab import drive
drive.mount('/gdrive')
import sys
sys.path.append('/gdrive/My Drive/ProteinsML/Protein-subcellular-localization/')
from models import CustomModels

Mounted at /gdrive


#Load the encoded protein sequences, labels and masks

In [5]:
# The masks are not needed for the FFN or CNN models
train = np.load('/gdrive/My Drive/ProteinsML/Protein-subcellular-localization/data/reduced_train.npz')
X_train = train['X_train']
y_train = train['y_train']
mask_train = train['mask_train']

validation = np.load('/gdrive/My Drive/ProteinsML/Protein-subcellular-localization/data/reduced_val.npz')
X_val = validation['X_val']
y_val = validation['y_val']
mask_val = validation['mask_val']

# One-hot encoding
n_class = 10
y_train = to_categorical(y_train, n_class)
y_val = to_categorical(y_val, n_class)

#Hyperparameter optimization

In [6]:
hp = kt.HyperParameters()

In [7]:
# fixed hyperparameters for all networks
batch_size = 128
seq_len = 400
n_feat = 20
n_class = 10
n_epochs = 120

# Searching for this hyperparameters:
n_hid=hp.Int('n_hid', min_value=10, max_value=60, step=10)
lr=hp.Float('lr', min_value=0.0001, max_value=0.0032, step=0.001)
drop_prob=hp.Float('drop_prob', min_value=0.1, max_value=0.7, step=0.2)
n_filt=hp.Int('n_filt', min_value=3, max_value=20, step=5)  # only present in convolutional layers



## Feed-Forward Network (FFN)



In [ ]:
# creating the model
FFN = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob)
# setting the optimizer 
FNN_tuner = kt.BayesianOptimization(FFN.create_FFN,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='FNN_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

FNN_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_FNN_hps = FNN_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 36s]
val_loss: 0.8260005116462708

Best val_loss So Far: 0.7702646851539612
Total elapsed time: 00h 17m 26s
INFO:tensorflow:Oracle triggered exit


In [ ]:
print(f"""
Results of FFN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_FNN_hps.get('n_hid')}
The optimal number of drop_prob is {best_FNN_hps.get('drop_prob')} 
the optimal learning rate for is {best_FNN_hps.get('lr')}.
""")


Results of FFN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is 10
The optimal number of drop_prob is 0.5000000000000001 
the optimal learning rate for is 0.0001.





## Convolutional Neural Network (CNN)



In [ ]:
# creating the model
CNN = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob, n_filt=n_filt)
# setting the optimizer 
CNN_tuner = kt.BayesianOptimization(CNN.create_CNN,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='CNN_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

CNN_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_CNN_hps = CNN_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 47s]
val_loss: 1.0201101303100586

Best val_loss So Far: 0.8649263978004456
Total elapsed time: 00h 24m 45s
INFO:tensorflow:Oracle triggered exit


In [ ]:
print(f"""
Results of CNN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_hps.get('lr')}.
""")


Results of CNN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is 10
The optimal number of n_filt is 3 
The optimal number of drop_prob is 0.7000000000000001 
the optimal learning rate for is 0.0031.




## Long Short Term Memory Network (LSTM)



In [ ]:
# creating the model
LSTM = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob)
# setting the optimizer 
LSTM_tuner = kt.BayesianOptimization(LSTM.create_LSTM,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='LSTM_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

LSTM_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_LSTM_hps = LSTM_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 05m 33s]
val_loss: 1.0160471200942993

Best val_loss So Far: 0.9618147611618042
Total elapsed time: 02h 44m 50s
INFO:tensorflow:Oracle triggered exit


In [ ]:
print(f"""
Results of LSTM optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_LSTM_hps.get('n_hid')}
The optimal number of drop_prob is {best_LSTM_hps.get('drop_prob')} 
the optimal learning rate for is {best_LSTM_hps.get('lr')}.
""")


Results of LSTM optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is 40
The optimal number of drop_prob is 0.7000000000000001 
the optimal learning rate for is 0.0011.




## Convolutional Neural Network with LSTM (CNN + LSTM)



In [8]:
# creating the model
CNN_LSTM = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob, n_filt=n_filt)
# setting the optimizer 
CNN_LSTM_tuner = kt.BayesianOptimization(CNN_LSTM.create_CNN_LSTM,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='CNN_LSTM_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

CNN_LSTM_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_CNN_LSTM_hps = CNN_LSTM_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 8 Complete [00h 18m 36s]
val_loss: 1.1222560405731201

Best val_loss So Far: 1.029266357421875
Total elapsed time: 03h 39m 20s

Search: Running Trial #9

Hyperparameter    |Value             |Best Value So Far 
n_hid             |10                |10                
lr                |0.0031            |0.0001            
drop_prob         |0.7               |0.7               
n_filt            |3                 |3                 

Epoch 1/120
76/76 [==============================] - 120s 2s/step - loss: 2.2988 - accuracy: 0.1035 - val_loss: 2.2879 - val_accuracy: 0.1858
Epoch 2/120
76/76 [==============================] - 112s 1s/step - loss: 2.2833 - accuracy: 0.1879 - val_loss: 2.2671 - val_accuracy: 0.2362
Epoch 3/120
76/76 [==============================] - 112s 1s/step - loss: 2.2583 - accuracy: 0.2377 - val_loss: 2.2343 - val_accuracy: 0.2378
Epoch 4/120
76/76 [==============================] - 112s 1s/step - loss: 2.2184 - accuracy: 0.2388 - val_loss: 2.1606 - val_acc

In [ ]:
print(f"""
Results of CNN + LSTM optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_LSTM_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_LSTM_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_LSTM_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_LSTM_hps.get('lr')}.
""")


## LSTM + Bahdanau Attention layer (LSTM + Attention)



In [ ]:
# creating the model
LSTM_Attention = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob)
# setting the optimizer 
LSTM_Attention_tuner = kt.BayesianOptimization(LSTM_Attention.create_LSTM_Attention,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='LSTM_Attention_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

LSTM_Attention_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_LSTM_Attention_hps = LSTM_Attention_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
Results of LSTM + Attention optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_LSTM_Attention_hps.get('n_hid')}
The optimal number of drop_prob is {best_LSTM_Attention_hps.get('drop_prob')} 
the optimal learning rate for is {best_LSTM_Attention_hps.get('lr')}.
""")



## CNN with LSTM and Attention layer (CNN + LSTM + Attention)



In [ ]:
# creating the model
CNN_LSTM_Attention = CustomModels(seq_len=seq_len, n_feat=n_feat, n_hid=n_hid, n_class=n_class, lr=lr, drop_prob=drop_prob, n_filt=n_filt)
# setting the optimizer 
CNN_LSTM_Attention_tuner = kt.BayesianOptimization(CNN_LSTM_Attention.create_CNN_LSTM_Attention,
                     hyperparameters=hp,
                     objective='val_loss',
                     max_trials=30,
                     directory='hyperOpt',
                     project_name='CNN_LSTM_Attention_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

CNN_LSTM_Attention_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, callbacks=[stop_early])

# Get the optimal hyperparameters
best_CNN_LSTM_Attention_hps = CNN_LSTM_Attention_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
Results of CNN + LSTM + Attention optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_LSTM_Attention_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_LSTM_Attention_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_LSTM_Attention_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_LSTM_Attention_hps.get('lr')}.
""")

# Results overview

In [ ]:
print(f"""
Results of FFN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_FNN_hps.get('n_hid')}
The optimal number of drop_prob is {best_FNN_hps.get('drop_prob')} 
the optimal learning rate for is {best_FNN_hps.get('lr')}.
----------------------------
""")

print(f"""
Results of CNN optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_hps.get('lr')}.
----------------------------
""")

print(f"""
Results of LSTM optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_LSTM_hps.get('n_hid')}
The optimal number of drop_prob is {best_LSTM_hps.get('drop_prob')} 
the optimal learning rate for is {best_LSTM_hps.get('lr')}.
----------------------------
""")

print(f"""
Results of CNN + LSTM optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_LSTM_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_LSTM_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_LSTM_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_LSTM_hps.get('lr')}.
----------------------------
""")

print(f"""
Results of LSTM + Attention optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_LSTM_Attention_hps.get('n_hid')}
The optimal number of drop_prob is {best_LSTM_Attention_hps.get('drop_prob')} 
the optimal learning rate for is {best_LSTM_Attention_hps.get('lr')}.
----------------------------
""")

print(f"""
Results of CNN + LSTM + Attention optimization
----------------------------
The hyperparameter search is complete. 
The optimal number of n_hid is {best_CNN_LSTM_Attention_hps.get('n_hid')}
The optimal number of n_filt is {best_CNN_LSTM_Attention_hps.get('n_filt')} 
The optimal number of drop_prob is {best_CNN_LSTM_Attention_hps.get('drop_prob')} 
the optimal learning rate for is {best_CNN_LSTM_Attention_hps.get('lr')}.
----------------------------
""")


>## Save files



In [13]:
# Save tuner hyperparameter
!zip -r ./hyperOpt.zip ./hyperOpt

updating: hyperOpt/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/tuner0.json (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/oracle.json (deflated 61%)
updating: hyperOpt/CNN_LSTM_tuner/trial_ec04ce4d909f1772fe900efe9835828f/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/trial_ec04ce4d909f1772fe900efe9835828f/trial.json (deflated 67%)
updating: hyperOpt/CNN_LSTM_tuner/trial_ec04ce4d909f1772fe900efe9835828f/checkpoints/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/trial_ec04ce4d909f1772fe900efe9835828f/checkpoints/epoch_0/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/trial_242cb4b99fdf39499cf0472e9c445b6b/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/trial_242cb4b99fdf39499cf0472e9c445b6b/trial.json (deflated 67%)
updating: hyperOpt/CNN_LSTM_tuner/trial_242cb4b99fdf39499cf0472e9c445b6b/checkpoints/ (stored 0%)
updating: hyperOpt/CNN_LSTM_tuner/trial_242cb4b99fdf39499cf0472e9c445b6b/checkpoints/epoch_0/ (stored 0%)
updating: hyperOpt/CNN_L

In [14]:
from google.colab import files
files.download("./hyperOpt.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>